# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,moussoro,13.6408,16.4901,78.66,13,95,9.98,TD,1675880515
1,1,la sarre,48.8002,-79.1996,16.79,89,99,6.17,CA,1675880515
2,2,petropavlovsk-kamchatskiy,53.0452,158.6483,-1.23,85,40,4.47,RU,1675880516
3,3,sioux lookout,50.1001,-91.9170,28.47,74,100,3.44,CA,1675880425
4,4,busselton,-33.6500,115.3333,59.72,88,0,11.34,AU,1675880517


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height =500,
    size = "Humidity",
    color = "City",
    scale = 0.6
)

# Display the map

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_temp = (city_data_df["Max Temp"] <= 60)
ideal_cloudcover = (city_data_df["Cloudiness"] <= 50)
ideal_wind = (city_data_df["Wind Speed"] <= 15)

#Humidity has been removed to increase the data set
#ideal_humidity = (city_data_df["Humidity"] <= 50)
              
ideal_weather = city_data_df.loc[(ideal_temp & ideal_wind & ideal_cloudcover),:]

# Drop any rows with null values
ideal_weather_data = ideal_weather.dropna()

# Display sample data
ideal_weather_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,petropavlovsk-kamchatskiy,53.0452,158.6483,-1.23,85,40,4.47,RU,1675880516
4,4,busselton,-33.6500,115.3333,59.72,88,0,11.34,AU,1675880517
40,40,laguna,38.4210,-121.4238,58.08,80,0,1.01,US,1675880560
42,42,saint george,37.1041,-113.5841,48.94,45,0,1.99,US,1675880561
53,53,anadyr,64.7500,177.4833,-28.70,75,0,8.95,RU,1675880565
...,...,...,...,...,...,...,...,...,...,...
495,495,asfi,32.2994,-9.2372,58.93,76,40,8.16,MA,1675880880
535,535,north adams,42.7009,-73.1087,42.76,56,40,11.50,US,1675880896
545,545,frauenfeld,47.5582,8.8985,30.94,76,0,1.03,CH,1675880900
546,546,mezen,65.8522,44.2400,20.26,92,41,9.35,RU,1675880901


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_data[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,petropavlovsk-kamchatskiy,RU,53.0452,158.6483,85,
4,busselton,AU,-33.6500,115.3333,88,
40,laguna,US,38.4210,-121.4238,80,
42,saint george,US,37.1041,-113.5841,45,
53,anadyr,RU,64.7500,177.4833,75,
...,...,...,...,...,...,...
495,asfi,MA,32.2994,-9.2372,76,
535,north adams,US,42.7009,-73.1087,56,
545,frauenfeld,CH,47.5582,8.8985,76,
546,mezen,RU,65.8522,44.2400,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    params["categories"] = f'accommodation.hotel'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
petropavlovsk-kamchatskiy - nearest hotel: ООО Постоялый двор
busselton - nearest hotel: Broadwater Beach Resort
laguna - nearest hotel: Holiday Inn Express & Suites
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
anadyr - nearest hotel: Гостевой дом
rawicz - nearest hotel: Sportowa
bahraich - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF - 3 Estrelas
mount gambier - nearest hotel: The Old Mount Gambier Gaol
upernavik - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
hofn - nearest hotel: Guesthouse Dyngja
beringovskiy - nearest hotel: No hotel found
yelizovo - nearest hotel: Николина Усадьба
yellowknife - nearest hotel: No hotel found
tuktoyaktuk - nearest hotel: Hunter's B&B
fort morgan - nearest hotel: Fairfield Inn & Suites Fort Morgan
lasa - nearest hotel: Hotel "Drei Kreuz"
groton - nearest hotel: Hampton Inn & Suites
qaanaaq - nearest hotel: Hotel Qaanaaq
kodiak - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
2,petropavlovsk-kamchatskiy,RU,53.0452,158.6483,85,ООО Постоялый двор
4,busselton,AU,-33.6500,115.3333,88,Broadwater Beach Resort
40,laguna,US,38.4210,-121.4238,80,Holiday Inn Express & Suites
42,saint george,US,37.1041,-113.5841,45,"The Advenire, an Autograph Collection Hotel"
53,anadyr,RU,64.7500,177.4833,75,Гостевой дом


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height =500,
    size = "Humidity",
    color = "City",
    scale = 0.6,
    hover_cols=["Hotel Name","Country"]    
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)